# MIMIC Pre-Processing Notebook

This notebook runs through the pre-processing we perform to get a single table of patient data.

MIMIC-III exists in a multi table format however our work is focused on generating synthetic versions of single table data. As a result we pre-process the data by collecting files from randomised patients and bring them into a single file format.

This file contains numerous continuous, categorical and datetime columns. It also has one column that contains missing data i.e. Date of Death information. This range can then be used to fully test data synthesis methods and ensuring they are robust to a realistic healthcare EHR type dataset.

This notebook can create various size tables. These different sizes are shown in cell headers. We can create a small, medium, large set each with 11k, 81k and 217k rows respectively. Run the cells to get the dataset size that you desire. The difference between them is mainly that we allow multiple entries for specific patients depending on how many times they are admitted to a respective hospital.

NOTE: Currently our model does NOT support missing data and as a result date of death is dropped and saved as a separate table for our analysis - future work should look at handling this common trend in data.

In [ ]:
import pandas as pd
import numpy as np
from random import randrange, seed

## Data Imports

Load in the compressed csv files that MIMIC-III provides. We choose a selection from these files however this is arbitrary. You can create whatever table you wish using our formula just note that it must follow the data guidance to work with the current version of SynthVAE.

In [ ]:
admissions = pd.read_csv('../data/admissions.csv.gz')
chartevents = pd.read_csv('../data/CHARTEVENTS.csv.gz',nrows=10000000)
icustays = pd.read_csv('../data/ICUSTAYS.csv.gz')
items = pd.read_csv('../data/D_ITEMS.csv.gz')
outputevents = pd.read_csv('../data/OUTPUTEVENTS.csv.gz')
patients = pd.read_csv('../data/PATIENTS.csv.gz')   

## Augment Patient DOBs

Here we augment patients DOBs in the MIMIC-III set. We are basically just bringing the date of births forward to closer mimic a period in time closer to when SynthVAE was created.

In [ ]:
seed(2021)
## Augment patient DOBs
new_dobs = []
dob_offset = []
for index, row in patients.iterrows():
    years_diff_behind = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='Y'))
    years_diff_ahead = len(pd.date_range(start=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),freq='Y'))

    if (years_diff_behind != 0) and (pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') < pd.to_datetime('1930-01-01 00:00:00',format='%Y-%m-%d %H:%M:%S')):
        num_years = randrange(years_diff_behind - 80, years_diff_behind - 40)
        new_dobs.append(pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(years=num_years))     
        dob_offset.append(num_years)
    elif (years_diff_ahead != 0) and (pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') > pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S')):
        num_years = randrange(years_diff_ahead + 30, years_diff_ahead + 50)
        new_dobs.append(pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') - pd.DateOffset(years=num_years))
        dob_offset.append(-num_years)
    else:
        new_dobs.append(pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'))
        dob_offset.append(0)
patients['DOB'] = new_dobs
patients['DOB_offset'] = dob_offset

## Produce small input (11k rows)

First we set the seed to get reproducible tables independent of user.

Secondly we merge all the respective tables we have chosen to form our single table as well as the columns that we desire.

Thirdly, we loop through the rows we are generating and add on the offset we added to the date of birth, in the cell above, to all the other relevant datetime columns.

Fourthly, we then ensure that the admit time is before the chart time and ensure that the datetime columns make sense in their given context.

Finally, we drop columns that are not required, reformat ages etc to make them appropriate all before saving the table.

In [ ]:
seed(2021)
# Generate small input data file
table_one = admissions[['ROW_ID','SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
table_one = table_one.merge(patients[['SUBJECT_ID','GENDER','DOB']])
table_one = table_one.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])
one_per_pat = chartevents.drop_duplicates(subset=['SUBJECT_ID','ICUSTAY_ID'])
table_one = table_one.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

for index, row in table_one.iterrows():
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25,0),np.round(admit_min*0.9,0)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01,0)+1,np.round(stay_len*0.99,0)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

table_one['ADMITTIME'] = new_admits
table_one['DISCHTIME'] = new_dischs
table_one['CHARTTIME'] = new_chart

table_one = table_one[(pd.to_datetime(table_one.ADMITTIME) < pd.to_datetime(table_one.CHARTTIME)) & (pd.to_datetime(table_one.DISCHTIME) > pd.to_datetime(table_one.CHARTTIME))]
table_one = table_one.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
table_one.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00'
table_one['DOB'] = pd.to_datetime(table_one['DOB'], format='%Y-%m-%d %H:%M:%S')
table_one['age'] = (age_calc - table_one['DOB']).astype('<m8[Y]')
table_one = table_one.groupby('SUBJECT_ID').head(4)
print(f"Small input table saved, number of columns:  {table_one.shape[1]}, number of rows: {table_one.shape[0]}")
#table_one.to_csv(f'data/05_model_input/table_one_{table_one.shape[0]}.csv')

In [ ]:
table_one.head()

## Produce medium input (81k rows)

The process to produce the table is exactly the same as the small table the only difference now is we add in an additional step inbetween the second and third steps.

This additional step chooses a data split where we are including multiple rows for that patient:

- Patients included in <b>split_one</b> are forced to only have one row in the dataset.

- Patients between <b>split_one</b> and <b>split_two</b> are forced to have only two rows in the dataset.

- Patients after <b>split_two</b> i.e. the remainder, are forced to have up to 100 rows in the dataset.

Note: Just because we specify <b>head(2)</b> or <b>head(100)</b> does NOT mean that every patient looped over will be given 2 or 100 rows in the single table. They will be given the amount of rows that they can feasibly have. I.e. if a patient in the middle set only has one row then <b>head(2)</b> is just going to put one row in the single table. Likewise <b>head(100)</b> for a patient with 5 rows, will only enter 5 rows into the single table. These values the rows a patient COULD have in the set provided they have that many rows available to us in MIMIC-III.

In [ ]:
seed(2021)
# Generate mid-sized input file
table_one = admissions[['ROW_ID','SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
table_one = table_one.merge(patients[['SUBJECT_ID','GENDER','DOB']])
table_one = table_one.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])

total_subjects = list(chartevents.SUBJECT_ID.unique())

df_list = []

split_one = int(np.round(len(total_subjects)*0.3,0))
split_two = int(np.round(len(total_subjects)*0.9,0))

for sub_index in range(0,split_one):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(1))

for sub_index in range(split_one,split_two):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(2))

for sub_index in range(split_two,len(total_subjects)):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(100))

one_per_pat = pd.concat(df_list)
one_per_pat.shape

table_one = table_one.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

patient_stays = table_one[['SUBJECT_ID','ICUSTAY_ID','DOB','ADMITTIME']]
patient_stays.drop_duplicates(inplace=True)

for index, row in table_one.iterrows():
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25,0),np.round(admit_min*0.9,0)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01,0)+1,np.round(stay_len*0.99,0)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

table_one['ADMITTIME'] = new_admits
table_one['DISCHTIME'] = new_dischs
table_one['CHARTTIME'] = new_chart

table_one = table_one[(pd.to_datetime(table_one.ADMITTIME) < pd.to_datetime(table_one.CHARTTIME)) & (pd.to_datetime(table_one.DISCHTIME) > pd.to_datetime(table_one.CHARTTIME))]
table_one = table_one.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
table_one.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00'
table_one['DOB'] = pd.to_datetime(table_one['DOB'], format='%Y-%m-%d %H:%M:%S')
table_one['age'] = (age_calc - table_one['DOB']).astype('<m8[Y]')
print(f"Mid sized input table saved, number of columns:  {table_one.shape[1]}, number of rows: {table_one.shape[0]}")
table_one.to_csv(f'data/05_model_input/table_one_imbalanced_{table_one.shape[0]}.csv')

## Produce large input (217k rows)

Same rules apply in here as the medium set, only difference here is that <b>split_one</b> and <b>split_two</b> are different sizes allowing for more patients with up to 2 and up to 100 rows.

In [ ]:
seed(2021)
# Generate large-sized input file
table_one = admissions[['ROW_ID','SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
table_one = table_one.merge(patients[['SUBJECT_ID','GENDER','DOB']])
table_one = table_one.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])

total_subjects = list(chartevents.SUBJECT_ID.unique())

df_list = []

split_one = int(np.round(len(total_subjects)*0.3,0))
split_two = int(np.round(len(total_subjects)*0.7,0))

for sub_index in range(0,split_one):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(1))

for sub_index in range(split_one,split_two):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(2))

for sub_index in range(split_two,len(total_subjects)):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(100))

one_per_pat = pd.concat(df_list)
one_per_pat.shape

table_one = table_one.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

patient_stays = table_one[['SUBJECT_ID','ICUSTAY_ID','DOB','ADMITTIME']]
patient_stays.drop_duplicates(inplace=True)

for index, row in table_one.iterrows():
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25,0),np.round(admit_min*0.9,0)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01,0)+1,np.round(stay_len*0.99,0)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

table_one['ADMITTIME'] = new_admits
table_one['DISCHTIME'] = new_dischs
table_one['CHARTTIME'] = new_chart

table_one = table_one[(pd.to_datetime(table_one.ADMITTIME) < pd.to_datetime(table_one.CHARTTIME)) & (pd.to_datetime(table_one.DISCHTIME) > pd.to_datetime(table_one.CHARTTIME))]
table_one = table_one.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
table_one.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00'
table_one['DOB'] = pd.to_datetime(table_one['DOB'], format='%Y-%m-%d %H:%M:%S')
table_one['age'] = (age_calc - table_one['DOB']).astype('<m8[Y]')
print(f"Large input table saved, number of columns:  {table_one.shape[1]}, number of rows: {table_one.shape[0]}")
table_one.to_csv(f'data/05_model_input/table_one_imbalanced_{table_one.shape[0]}.csv')